📘 Kalenderbasierte Features: Feiertage

In [1]:
from workalendar.europe import Germany
import pandas as pd

# Daten laden
base_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/"
df_umsatz = pd.read_csv(base_url + "umsatzdaten_gekuerzt.csv")
df_wetter = pd.read_csv(base_url + "wetter.csv")
df_kiwo = pd.read_csv(base_url + "kiwo.csv")

# Datum konvertieren
df_umsatz['Datum'] = pd.to_datetime(df_umsatz['Datum'])
df_wetter['Datum'] = pd.to_datetime(df_wetter['Datum'])
df_kiwo['Datum'] = pd.to_datetime(df_kiwo['Datum'])

# Merge & Features
df = df_umsatz.merge(df_wetter, on='Datum', how='left')
df = df.merge(df_kiwo, on='Datum', how='left')
df['KielerWoche'] = df['KielerWoche'].fillna(0).astype(int)
df['Wochentag'] = df['Datum'].dt.dayofweek
df['Monat'] = df['Datum'].dt.month
df['IstWochenende'] = df['Wochentag'].isin([5, 6]).astype(int)

📆 Feiertage berechnen:

In [2]:
# Feiertage in Schleswig-Holstein berechnen
cal = Germany(subdivision="SH")
jahre = df['Datum'].dt.year.unique()
feiertage_set = set()
for jahr in jahre:
    feiertage_set.update([d for d, name in cal.holidays(jahr)])

# Neue Spalte "Feiertag" ergänzen
df['Feiertag'] = df['Datum'].isin(feiertage_set).astype(int)

/var/folders/09/wpbg8c7x22x3d_2_ybk_s1g40000gn/T/ipykernel_66930/2533174572.py:9: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['Feiertag'] = df['Datum'].isin(feiertage_set).astype(int)


🔍 Kontrolle:

In [3]:
# Prüfung der ersten 10 Zeilen
df[['Datum', 'Feiertag']].drop_duplicates().sort_values('Datum').head(10)

,Datum,Feiertag
0,2013-07-01,0
1,2013-07-02,0
2,2013-07-03,0
3,2013-07-04,0
4,2013-07-05,0
5,2013-07-06,0
6,2013-07-07,0
7,2013-07-08,0
8,2013-07-09,0
9,2013-07-10,0


# Schulferien hinzufügen
📥 CSV einlesen und verarbeiten

In [6]:
# Ferien-CSV einlesen
df_ferien = pd.read_csv("../../data/ferien_sh.csv", parse_dates=["Startdatum", "Enddatum"])

# Alle einzelnen Ferientage als Set extrahieren
ferien_tage = set()
for _, row in df_ferien.iterrows():
    tage = pd.date_range(start=row["Startdatum"], end=row["Enddatum"])
    ferien_tage.update(tage)

# Neue Spalte erstellen
df['Ferienzeit'] = df['Datum'].isin(ferien_tage).astype(int)

🔍 Kontrolle:

In [7]:
df[['Datum', 'Ferienzeit']].drop_duplicates().sort_values('Datum').head(10)

,Datum,Ferienzeit
0,2013-07-01,1
1,2013-07-02,1
2,2013-07-03,1
3,2013-07-04,1
4,2013-07-05,1
5,2013-07-06,1
6,2013-07-07,1
7,2013-07-08,1
8,2013-07-09,1
9,2013-07-10,1
